In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from msr.data.dataset_providers import SleepEDFDatasetProvider, PtbXLDatasetProvider, MimicDatasetProvider
from msr.data.download import mimic, sleep_edf, ptbxl
from msr.data.namespace import Representation, PeriodicRepresentation
import torch
import numpy as np

In [23]:
sleepedf_ds_provider = SleepEDFDatasetProvider(
    representation_types=Representation.all(),
    windows_params=dict(win_len_s=10, step_s=8)
)

ptbxl_ds_provider = PtbXLDatasetProvider(
    representation_types=PeriodicRepresentation.all(),
    windows_params=dict(win_len_s=6, step_s=2),
    beats_params=dict(source_channel="ecg_2"),
    agg_beat_params=dict(valid_only=False),
    n_beats=7,
    target="diagnostic_class"
)

mimic_ds_provider = MimicDatasetProvider(
    representation_types=PeriodicRepresentation.all(),
    windows_params=dict(win_len_s=20, step_s=18),
    beats_params=dict(source_channel="ecg"),
    agg_beat_params=dict(valid_only=False),
    n_beats=100,
    target="sbp_dbp_avg"
)

In [ ]:
sleepedf_sample = torch.load(sleep_edf.RAW_TENSORS_DATA_PATH / "train.pt")[0]
ptbxl_sample = torch.load(ptbxl.RAW_TENSORS_DATA_PATH / "train.pt")[0]
mimic_sample = torch.load(mimic.RAW_TENSORS_DATA_PATH / "train.pt")[0]

sleepedf_reps, sleepedf_feat_names = sleepedf_ds_provider.get_representations(sleepedf_sample.numpy(), return_feature_names=True)
# sleepedf_reps, sleepedf_feat_names

ptbxl_reps, ptbxl_feat_names = ptbxl_ds_provider.get_representations(ptbxl_sample.numpy().T, return_feature_names=True)
# ptbxl_reps, ptbxl_feat_names

mimic_reps, mimic_feat_names = mimic_ds_provider.get_representations(mimic_sample.numpy().T, return_feature_names=True)
# mimic_reps, mimic_feat_names

In [24]:
ds = mimic_ds_provider.load_split("train", "whole_signal_features")

No info file for that split


In [25]:
ds

{'data': tensor([[1.7923, 0.6548, 1.6828,  ..., 0.0496, 1.3955, 0.0000],
         [1.7458, 0.5886, 1.5098,  ..., 0.0043, 1.4540, 0.0000],
         [1.7933, 0.6552, 1.5064,  ..., 0.0039, 1.4231, 0.0000],
         ...,
         [0.4836, 0.1227, 0.4848,  ..., 0.0825, 0.6428, 0.0000],
         [0.4891, 0.2269, 0.5239,  ..., 0.0032, 0.3323, 0.0000],
         [0.4963, 0.0735, 0.4985,  ..., 0.0636, 0.3699, 0.0000]],
        dtype=torch.float64),
 'targets': tensor([[144.5071,  77.4045],
         [140.5207,  65.6062],
         [159.5910,  76.0887],
         ...,
         [ 86.7306,  38.0804],
         [ 99.7970,  49.6916],
         [114.4846,  47.7820]], dtype=torch.float64),
 'info': {},
 'feature_names': array(['ppg__basic__mean', 'ppg__basic__std', 'ppg__basic__median',
        'ppg__basic__skewness', 'ppg__basic__kurtosis',
        'ppg__agg_beat__basic__mean', 'ppg__agg_beat__basic__std',
        'ppg__agg_beat__basic__median', 'ppg__agg_beat__basic__skewness',
        'ppg__agg_beat__bas

In [27]:
ptbxl_ds_provider.create_dataset()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [4]:
sleepedf_ds_provider.create_dataset(splits=["val", "test"])

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [5]:
mimic_ds_provider.create_dataset()

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [20]:
ds_name = "ptbxl"
rep_type = "whole_signal_features"
split = "train"
rep_suffix = "_100" if ds_name == "ptbxl" else ""
old_path = f"../data/{ds_name}/representations{rep_suffix}/{rep_type}/{split}.pt"
new_path = f"../data/{ds_name}/representations{rep_suffix}/{rep_type}/_{split}.pt"

torch.all(torch.load(old_path)[:5] == torch.load(new_path))

tensor(True)

In [1]:
import torch
import numpy as np

In [2]:
from msr.data.download import ptbxl

In [5]:
classname = "diagnostic_class" # "diagnostic_subclass"

In [7]:
train_targets = np.load(ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "train.npy")
val_targets = np.load(ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "val.npy")
test_targets = np.load(ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "test.npy")

torch.save(torch.from_numpy(train_targets), ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "train.pt")
torch.save(torch.from_numpy(val_targets), ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "val.pt")
torch.save(torch.from_numpy(test_targets), ptbxl.RAW_TENSORS_TARGETS_PATH / classname / "test.pt")